In [1]:
import os
import sys
import netCDF4
import shutil
import tempfile
import pandas as pd
import numpy as np

In [2]:
!pwd

/OSM/CBR/OA_GLOBALCABLE/work/BIOS3_simulations/python_scripts/python_notebooks_Vanessa


# Edit Site Met Files

In [3]:
site_met_path = '/OSM/CBR/OA_GLOBALCABLE/work/BIOS3_forcing/site_met/'

In [4]:
#files=['Warra_met_2014_2017.nc', 'AU_Cum_2014_2017_met.nc', 'Tumbarumba_met_2002_2016.nc', 'Wombat_met_2010_2018_iveg.nc']
files=['HowardFluxnet.1.4_met.nc']

In [5]:
iveg_vals = np.array([(2,7)])
patchfrac_vals = np.array([(0.5,0.5)])

In [9]:
def put_tile_info(met_fname=None, new_met_fname="tmp.nc"): 
    shutil.copyfile(met_fname, new_met_fname)
    nc = netCDF4.Dataset(new_met_fname, 'r+')
    #(nc_attrs, nc_dims, nc_vars) = ncdump(nc)
    nc.createDimension("patch", 2)  
    iveg = nc.createVariable("iveg","i4",("patch","y","x"))
    iveg[:] = [2,7]
    patchfrac = nc.createVariable("patchfrac","f4",("patch","y","x"))
    patchfrac[:] = [0.5, 0.5]
    nc.close()

In [10]:
i=0
for f in files:
    met_fname = site_met_path + f
    print(met_fname)
    met_fname_new = met_fname[:-3] + '_tiled.nc'
    put_tile_info(met_fname=met_fname, new_met_fname=met_fname_new)
    i=i+1

/OSM/CBR/OA_GLOBALCABLE/work/BIOS3_forcing/site_met/HowardFluxnet.1.4_met.nc


In [ ]:
def  set_Tair(met_fname=None, new_met_fname="tmp.nc"): 
    shutil.copyfile(met_fname, new_met_fname)
    nc = netCDF4.Dataset(new_met_fname, 'r+')
    (nc_attrs, nc_dims, nc_vars) = ncdump(nc)
    Tair = nc.variables['Tair'][:,0,0]
    nc.variables['Tair'][:,0,0] = Tair.mean()
    nc.close()  # close the new file
    print(Tair.mean())
    return new_met_fname

In [ ]:
def ncdump(nc_fid):
    '''
    ncdump outputs dimensions, variables and their attribute information.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions

    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables

    return nc_attrs, nc_dims, nc_vars

In [ ]:
def change_LAI(met_fname=None, fixed=None, lai_fname=None, new_met_fname="tmp.nc"):

    if fixed is not None:
        lai = fixed
    else:
        lai = pd.read_csv(lai_fname)

    shutil.copyfile(met_fname, new_met_fname)

    nc = netCDF4.Dataset(new_met_fname, 'r+')
    (nc_attrs, nc_dims, nc_vars) = ncdump(nc)

    try:
        nc_var = nc.createVariable('LAI', 'f4', ('time', 'y', 'x'))
    except:
        print('variable exists')
        pass
   
    nc.setncatts({'long_name': "Leaf Area Index",})
    nc.variables['LAI'][:,0,0] = lai
    nc.close()  # close the new file

    return new_met_fname